In [18]:
import numpy as np
import imutils
import cv2
import os
import time
import requests


## RCNN + Grabcut method.

### Load pretrained labels, weights and model 

In [20]:
label = os.path.join("helper", "object_detection_classes_coco.txt")
LABELS = open(label).read().strip().split("\n")
weightsPath = os.path.join("helper", "frozen_inference_graph.pb")
configPath = os.path.join("helper","mask_rcnn_inception_v2_coco_2018_01_28.pbtxt")
print("Loaded model.")
net = cv2.dnn.readNetFromTensorflow(weightsPath, configPath)

Loaded model.


In [21]:
# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

### Read input image

In [22]:
image = cv2.imread("Image1.jpg")
cv2.imshow("Input", image)
cv2.waitKey()

-1

### Code to calculate Dice score.

In [23]:
def compute_dice_coefficient(mask_gt, mask_pred):
  volume_sum = mask_gt.sum() + mask_pred.sum()
  if volume_sum == 0:
    return np.NaN
  volume_intersect = (mask_gt & mask_pred).sum()
  return 2*volume_intersect / volume_sum 

### Do predictions.

In [24]:
blob = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
net.setInput(blob)
(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])

##### Threshold value is obtained by experiments, more threshold tighter requires more confidence. 

In [25]:
threshold = 0.05
outputMask = None
for i in range(0, boxes.shape[2]):
    classID = int(boxes[0, 0, i, 1])
    confidence = boxes[0, 0, i, 2]
    
    if confidence > threshold:
        print("Class detected is {}".format(LABELS[classID]))
        
        (H, W) = image.shape[:2]
        box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
        (startX, startY, endX, endY) = box.astype("int")
        boxW = endX - startX
        boxH = endY - startY
        mask = masks[i, classID]
        mask = cv2.resize(mask, (boxW, boxH),  interpolation=cv2.INTER_CUBIC)
        mask = (mask > threshold).astype("uint8") * 255
        
        rcnnMask = np.zeros(image.shape[:2], dtype="uint8")
        rcnnMask[startY:endY, startX:endX] = mask
        
        
        
        cv2.imshow("R-CNN Mask Intermediate result", rcnnMask)
        cv2.waitKey(0)
        
        temp = rcnnMask.copy()
        temp[temp > 0] = cv2.GC_PR_FGD
        temp[temp == 0] = cv2.GC_BGD
        
        # Apply grab cut now
        fgModel = np.zeros((1, 65), dtype="float")
        bgModel = np.zeros((1, 65), dtype="float")
        (temp, bgModel, fgModel) = cv2.grabCut(image, temp, None, bgModel, fgModel, iterCount=5, mode=cv2.GC_INIT_WITH_MASK)
        
        outputMask = np.where((temp == cv2.GC_BGD) | (temp == cv2.GC_PR_BGD), 0, 1)
        outputMask = (outputMask * 255).astype("uint8")
        #cv2.imwrite("Image2_seg.png", outputMask)
        cv2.imshow("GrabCut Mask", outputMask)
        cv2.waitKey(0)

Class detected is cow


## Below is code is for YOLO + grabcut method.

In [12]:
def detectObject(image):
    yolo_config='yolov3.cfg'
    if not os.path.isfile(yolo_config):
        url='https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg'
        r=requests.get(url)
        with open(yolo_config,'wb') as f:
            f.write(r.content)
    yolo_weights = 'yolov3.weights'
    if not os.path.isfile(yolo_weights):
        url = 'https://pjreddie.com/media/files/yolov3.weights'
        r = requests.get(url)
        with open(yolo_weights, 'wb') as f:
            f.write(r.content)        
    classes_file='coco.names'
    if not os.path.isfile(classes_file):
        url='https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names'
        r=requests.get(url)
        with open(classes_file,'wb') as f:
            f.write(r.content)
    
    with open(classes_file,'r') as f:
        classes=[line.strip() for line in f.readlines()]

   
    blob=cv2.dnn.blobFromImage(image,1/255,(416,416),(0,0,0),True,crop=False)
    net=cv2.dnn.readNet(yolo_weights,yolo_config)
    net.setInput(blob)
    layer_names=net.getLayerNames()
    output_layers=[layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    outs=net.forward(output_layers)
    class_ids= list()
    confidences= list()
    boxes= list()

    for out in outs:
        # iterate over anchor boxes for human class
        for detection in out:
            #bounding box
            center_x=int(detection[0] * image.shape[1])
            center_y=int(detection[1] * image.shape[0])
            w=int(detection[2] * image.shape[1])
            h=int(detection[3] * image.shape[0])
            x=center_x - w // 2
            y=center_y - h // 2
            boxes.append([x,y,w,h])
            #class
            class_id=np.argmax(detection[5:])
            class_ids.append(class_id)
            confidence=detection[4]
            confidences.append(float(confidence))
    #non-max supression
    ids=cv2.dnn.NMSBoxes(boxes,confidences,score_threshold=0.2,nms_threshold=0.9)
    
    colors= np.random.uniform(0,255,size=(len(classes),3))
    for i in ids:
        i=i[0]
        x,y,w,h=boxes[i]
        class_id=class_ids[i]
        color=colors[class_id]
        return (x, y, x+h, y+w)
        
    return None


In [13]:
rec = detectObject(image)

# If object not found then use complete image.
if rec == None:
    rec = (0, 0, image.shape[0], image.shape[1])
print(rec)

(45, 58, 279, 328)


### Apply grab cut.

In [14]:
mask = np.zeros(image.shape[:2], dtype="uint8")
fgModel = np.zeros((1, 65), dtype="float")
bgModel = np.zeros((1, 65), dtype="float")
# apply GrabCut using the the bounding box segmentation method
start = time.time()
(mask, bgModel, fgModel) = cv2.grabCut(image, mask, rec, bgModel,fgModel, iterCount=5, mode=cv2.GC_INIT_WITH_RECT)
end = time.time()
print("GrabCut took {:.2f} seconds".format(end - start))

GrabCut took 0.89 seconds


### Extract foreground mask

In [15]:
values = (
    ("Definite Background", cv2.GC_BGD),
    ("Probable Background", cv2.GC_PR_BGD),
    ("Definite Foreground", cv2.GC_FGD),
    ("Probable Foreground", cv2.GC_PR_FGD),
)
outputMask = (mask == cv2.GC_PR_FGD).astype("uint8") * 255
#cv2.imwrite("Image1_seg.png", outputMask)
cv2.imshow("Probable Foreground", outputMask)
cv2.waitKey(0)

-1

### Run  below cell to calculate dice score.

In [16]:

mask_gt = cv2.imread("Image1_GT.png")
image_pred = cv2.imread("Image1_seg.png")
compute_dice_coefficient(mask_gt, image_pred)


0.9615456890513692